## What is StaticFrame

## DataFrames Beyond Pandas: An Introduction to StaticFrame

## StaticFrame for Pandas Users




* History
    * Motivation
        * Spooky actions at a distance
        * Redundancies, inconsistencies
        * NumPy datetime64, strings
    * Development
    
* Current State
    * Code, Docs, Packages
    * Quality & Test
    * Contributing

* The Frame and the Series
    * DataFrame as labelled arrays
    * Focus on 2D and 1d
    containers
    * nD via hierarchical indices, higher-order containers

* Introspective containers
    * interface attrubutes
    * Using the docs
    
* A more explicit interface
    * keyword arguments everywhere
    * Orthogonal parameters
    * examples
        * rank v. rank_ordinal
        * join v. join_left
        * rename, relabel
        * iter_element().apply(), map_all(), map_fill(), map_any()

* Immutability and "no-copy" operations
    * Immutability reduces opportunities for errors
    * Immutability means we can pass around views without defensive copies

* Getting data in and out
    * Consistent from / to naming
    * From in-memory objects
    * From files
        * NPZ: fastest and most compatible

* More than one type of Frame
    * Frame, FrameGO, FrameHE

* More than one type of Index
    * Index, IndexDate, IndexYear, etc.
    * IndexHierarchy

* Renaming, reindexing, and relabeling
    * name attributes
    * reindexing
    * relabeling

* Iteration and function application

* Grouping and Windowing

* Higher-Order Containers

* Performance


